## Phase 3 (Data Preparation):  v4: (Int)

* Autorin: Anna (i3-Versicherung)
* Webseite: [Data Science Training - Kapitel 5](https://data-science.training/kapitel-5/)
* Datum: 23.03.2023

Wir führen eine verbesserte Datenvorbereitung durch.

* Der Cabin-Wert zur PassengerId 873 wird gelöscht
* Neues Feature: KnownCabin (1 = Kabine ist bekannt, 0 = Kabine ist unbekannt [Cabin: Missing Value bzw. NaN])
* Neues Feature: Child (Age < 12)
* Schätzen von Embarked Missing Values: Modus 'S'
* Schätzen von Fare Missing Values: Konstanter Wert 7,896
* Neues Feature: Title (aus Name)
* Neues Feature: FamilySizeBinned (aus SibSp und Parch)
* Neues Feature: FareBinned (aus Fare)

Unser Ziel von Version 4 ist es, Attribute (bzw. Features) zu erstellen, die nur die Werte 0 und 1 enthalten, also vom Datentyp Int sind (Boolean).

Hierzu benutzen wir One Hot Encoding, um sogenannte Dummy-Variablen als neue Features zu erzeugen.

* One Hot Encoding: Pclass, Sex, Embarked, Title, FamilySizeBinned, FareBinned => binäre Dummy-Variablen

Mit Hilfe einer Korrelationsanalyse werden wir Features filtern, deren Korrelationskoeffizient die Schwelle von 0,75 überschreitet. In diesem Fall sind die Features stark abhängig voneinander.

In [4]:
# Pandas Paket (Package) importieren
#  Datenstrukturen und Datenanalyse, I/O
#  https://pandas.pydata.org/pandas-docs/stable/
import pandas as pd
# NumPy Paket (Package) importieren
#  Mehrdimensionale Datenstrukturen (Vektoren, Matrizen, Tensoren, Arrays), Lineare Algebra
#  https://numpy.org/doc/
import numpy as np
# Eigene Module importieren
#  zur Berechnung der Korrelationskoeffizienten
import sys
sys.path.append('../00_DST_Module/')
import dst_correlation_functions as cf

In [5]:
# Trainings- und Testdaten als Pandas Data Frame (df) aus CSV-Dateien laden
#  (KNIME: "CSV Reader")
df_train = pd.read_csv('../../data/titanic/original/train.csv')
df_test  = pd.read_csv('../../data/titanic/original/test.csv')

In [6]:
# Trainings- und Testdaten zusammenführen
#  (KNIME "Concatenate")
df = pd.concat([df_train, df_test], ignore_index=True)

In [7]:
# Datentypen automatisch konvertieren
df = df.convert_dtypes()

In [8]:
# Fehlende Werte prüfen
df.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [9]:
# Falsche Kabinennummer löschen
#  (KNIME: "Rule Engine")
display(df[df['Cabin'] == 'B51 B53 B55'])
display(df[df['PassengerId'] == 873])
df.loc[872, 'Cabin'] = np.nan
display(df[df['PassengerId'] == 873])

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
679,680,1,1,"Cardeza, Mr. Thomas Drake Martinez",male,36.0,0,1,PC 17755,512.3292,B51 B53 B55,C
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0,B51 B53 B55,S
1234,1235,<NA>,1,"Cardeza, Mrs. James Warburton Martinez (Charlo...",female,58.0,0,1,PC 17755,512.3292,B51 B53 B55,C


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0,B51 B53 B55,S


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0,<NA>,S


In [10]:
# Neues Feature: KnownCabin
#  (KNIME: "Rule Engine")
df['KnownCabin'] = (df['Cabin'].notna()).astype('int')

In [11]:
# Neues Feature: Child
#  (KNIME: "Rule Engine")
df['Child'] = (df['Age'] < 12).fillna(False).astype('int')

In [12]:
# Fehlende Werte behandeln (d.h. schätzen) - Teil 1
#  (KNIME: "Missing Values")
# Embarked (Nominalskala): 2 fehlende Werte => Benutze den Modus (häufigster Wert)
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].dropna().mode()[0])

In [13]:
# Fehlende Werte behandeln (d.h. schätzen) - Teil 2
#  (KNIME: "Missing Values")
# Fare (Kardinalskala): 1 fehlender Wert => Benutze den konstanten Wert 7,896
#df['Fare'] = df['Fare'].fillna(7.896)
display(df[df['Fare'].isna()])
df.loc[1043, 'Fare'] = 7.896
display(df[df['PassengerId'] == 1044])

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,KnownCabin,Child
1043,1044,<NA>,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,<NA>,<NA>,S,0,0


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,KnownCabin,Child
1043,1044,<NA>,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,7.896,<NA>,S,0,0


In [14]:
# Hinweis: Wir löschen später die Features Age und Cabin.
# Deshalb ersetzen wir zu diesen Features keine fehlenden Werte.

In [15]:
# Neues Feature Title
#  (KNIME: "Cell Splitter", "Column Renamer", "Table Creator", "Cell Replacer")
df['Title'] = df['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]
df['Title'] = df['Title'].replace(['Ms', 'Mlle'], 'Miss')
df['Title'] = df['Title'].replace(['Mme', 'Lady', 'the Countess', 'Dona'], 'Mrs')
df['Title'] = df['Title'].replace(['Dr', 'Col', 'Major', 'Jonkheer', 'Capt', 'Sir', 'Don', 'Rev'], 'Rare')

In [16]:
# Neues Feature FamilySizeBinned
#  (KNIME: "Math Formula", "Table Creator", "Binner (Dictionary)")
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
bins   = [0, 2, 5, 99]
labels = ['No', 'Small', 'Large']
df['FamilySizeBinned'] = pd.cut(df['FamilySize'], bins, right=False, labels=labels)

In [17]:
# Neues Feature FareBinned
#  (KINME: "Table Creator", "Binner (Dictionary)")
bins   = [-1, 8, 16, 32, 1024]
labels = ['Low', 'Medium', 'High', 'VeryHigh']
df['FareBinned'] = pd.cut(df['Fare'], bins, right=False, labels=labels)

In [18]:
# Ergebnis des Feature Engineering anzeigen
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   PassengerId       1309 non-null   Int64   
 1   Survived          891 non-null    Int64   
 2   Pclass            1309 non-null   Int64   
 3   Name              1309 non-null   string  
 4   Sex               1309 non-null   string  
 5   Age               1046 non-null   Float64 
 6   SibSp             1309 non-null   Int64   
 7   Parch             1309 non-null   Int64   
 8   Ticket            1309 non-null   string  
 9   Fare              1309 non-null   Float64 
 10  Cabin             294 non-null    string  
 11  Embarked          1309 non-null   string  
 12  KnownCabin        1309 non-null   int32   
 13  Child             1309 non-null   int32   
 14  Title             1309 non-null   string  
 15  FamilySize        1309 non-null   Int64   
 16  FamilySizeBinned  1309 n

In [19]:
# One Hot Encoding => Dummy-Variablen
#  für Pclass, Sex, Embarked, Title, FamilySizeBinned, FareBinned
cols  = ['Pclass', 'Sex', 'Embarked', 'Title', 'FamilySizeBinned', 'FareBinned']
df = pd.get_dummies(df, columns=cols, dtype=int)
# Ergebnis anzeigen
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   PassengerId             1309 non-null   Int64  
 1   Survived                891 non-null    Int64  
 2   Name                    1309 non-null   string 
 3   Age                     1046 non-null   Float64
 4   SibSp                   1309 non-null   Int64  
 5   Parch                   1309 non-null   Int64  
 6   Ticket                  1309 non-null   string 
 7   Fare                    1309 non-null   Float64
 8   Cabin                   294 non-null    string 
 9   KnownCabin              1309 non-null   int32  
 10  Child                   1309 non-null   int32  
 11  FamilySize              1309 non-null   Int64  
 12  Pclass_1                1309 non-null   int32  
 13  Pclass_2                1309 non-null   int32  
 14  Pclass_3                1309 non-null   

### Zwischenergebnis

Wir haben viele neue Features erstellt. Nun werden wir die Attribute herausfiltern, die als Basis für diese neuen Features benutzt wurden. Also:

* Name   (wird durch Title ersetzt)
* Age    (wird durch Child ersetzt und hat außerdem fehlende Werte)
* SibSp  (wird durch FamilySize ersetzt)
* Parch  (wird durch FamilySize ersetzt)
* Fare   (wird durch FareBinned ersetzt)
* Cabin  (wird durch KnownCabin ersetzt und hat außerdem fehlende Werte)

Für ein neu gebildetes Feature gilt das aber auch:

* FamilySize (wird durch FamilySizeBinned bzw. deren Dummy-Variablen ersetzt)

Schließlich hat das Attribut Ticket keine Bedeutung für die Datenanalyse, es wird also als irrelevant betrachtet und kann ebenfalls herausgefiltert werden:

* Ticket (irrelevant)

In [21]:
# Aufräumen: Attribute (manuell) herausfiltern
#  (KNIME "Column Filter")
df = df.drop(['Name', 'Age', 'SibSp', 'Parch', 'Fare' , 'Cabin', 'FamilySize', 'Ticket'], axis=1)
# Ergebnis anzeigen
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   PassengerId             1309 non-null   Int64
 1   Survived                891 non-null    Int64
 2   KnownCabin              1309 non-null   int32
 3   Child                   1309 non-null   int32
 4   Pclass_1                1309 non-null   int32
 5   Pclass_2                1309 non-null   int32
 6   Pclass_3                1309 non-null   int32
 7   Sex_female              1309 non-null   int32
 8   Sex_male                1309 non-null   int32
 9   Embarked_C              1309 non-null   int32
 10  Embarked_Q              1309 non-null   int32
 11  Embarked_S              1309 non-null   int32
 12  Title_Master            1309 non-null   int32
 13  Title_Miss              1309 non-null   int32
 14  Title_Mr                1309 non-null   int32
 15  Title_Mrs            

In [22]:
# Version 4: Datentyp int, Ausnahme Attribut Survived
#df['Survived'] = df['PassengerId'].astype('int') # Ausnahme: Primärschlüsselattribut
# Ergebnis anzeigen
#df.info()

In [23]:
# Korrelationen: Numerische Attribute
corr_matrix = cf.dst_correlation_matrix(df)
display(corr_matrix)
#
corr_measures = cf.dst_correlation_measures_filtered(corr_matrix)
display(corr_measures)

,PassengerId,Survived,KnownCabin,Child,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,...,Title_Mr,Title_Mrs,Title_Rare,FamilySizeBinned_No,FamilySizeBinned_Small,FamilySizeBinned_Large,FareBinned_Low,FareBinned_Medium,FareBinned_High,FareBinned_VeryHigh
PassengerId,1.000000,-0.005007,-0.001264,-0.033648,0.026495,0.022714,-0.041544,-0.013406,0.013406,0.048101,...,0.014116,0.036282,-0.005303,0.028546,0.002975,-0.063415,0.028169,-0.018852,-0.028927,0.018074
Survived,-0.005007,1.000000,0.319572,0.112119,0.285904,0.093349,-0.322308,0.543351,-0.543351,0.168240,...,-0.549199,0.347864,-0.031348,-0.203367,0.279855,-0.125147,-0.210437,-0.094687,0.076704,0.244339
KnownCabin,-0.001264,0.319572,1.000000,-0.053539,0.774726,-0.175749,-0.526260,0.138923,-0.138923,0.259501,...,-0.133529,0.176986,0.067705,-0.177508,0.212918,-0.056032,-0.287059,-0.241127,-0.029179,0.581303
Child,-0.033648,0.112119,-0.053539,1.000000,-0.121609,0.020174,0.088684,0.060249,-0.060249,-0.027987,...,-0.314010,-0.116419,-0.038911,-0.324950,0.182293,0.301205,-0.161952,-0.017647,0.199392,-0.004378
Pclass_1,0.026495,0.285904,0.774726,-0.121609,1.000000,-0.296526,-0.622172,0.107371,-0.107371,0.325722,...,-0.099725,0.154347,0.121721,-0.126551,0.165965,-0.067523,-0.321473,-0.348468,0.039468,0.663072
Pclass_2,0.022714,0.093349,-0.175749,0.020174,-0.296526,1.000000,-0.563180,0.028862,-0.028862,-0.134675,...,-0.038595,0.064672,0.060296,-0.035075,0.097270,-0.118495,-0.294595,0.269863,0.164084,-0.131007
Pclass_3,-0.041544,-0.322308,-0.526260,0.088684,-0.622172,-0.563180,1.000000,-0.116562,0.116562,-0.171430,...,0.117925,-0.186564,-0.154747,0.138250,-0.223338,0.155560,0.519648,0.080292,-0.168658,-0.466329
Sex_female,-0.013406,0.543351,0.138923,0.060249,0.107371,0.028862,-0.116562,1.000000,-1.000000,0.066564,...,-0.870678,0.572861,-0.094406,-0.284537,0.255196,0.077748,-0.173201,-0.057553,0.070745,0.173511
Sex_male,0.013406,-0.543351,-0.138923,-0.060249,-0.107371,-0.028862,0.116562,-1.000000,1.000000,-0.066564,...,0.870678,-0.572861,0.094406,0.284537,-0.255196,-0.077748,0.173201,0.057553,-0.070745,-0.173511
Embarked_C,0.048101,0.168240,0.259501,-0.027987,0.325722,-0.134675,-0.171430,0.066564,-0.066564,1.000000,...,-0.065538,0.107572,0.022153,-0.107874,0.159594,-0.092825,-0.082218,-0.102088,-0.052162,0.243330


Sex_female           Sex_male                  1.000000
FamilySizeBinned_No  FamilySizeBinned_Small    0.873398
Sex_female           Title_Mr                  0.870678
Embarked_C           Embarked_S                0.778262
KnownCabin           Pclass_1                  0.774726
dtype: float64

### Schlussfolgerungen

Zwischen einigen Attributen (bzw. Features) gibt es starke Zusammenhänge:

* Sex_female           und Sex_male
* Sex_female           und Title_Mr
* FamilySizeBinned_No  und FamilySizeBinned_Small
* Embarked_C           und Embarked_S
* KnownCabin           und Pclass_1

Wir können also jeweils eines dieser Feature eliminieren. Wir können beispielsweise die Features der linken Seite behalten:

* Sex_female
* FamilySizeBinned_No
* Embarked_C
* KnownCabin

Und entsprechend können wir die Features der rechten Seite herausfiltern:

* Sex_male
* Title_Mr
* FamilySizeBinned_Small
* Embarked_S
* Pclass_1

Prinzipiell wäre aber auch eine Mischung möglich. In der KNIME Analytics Platform werden beispielsweise die folgenden Features herausgefiltert:

* Sex_female
* Title_Mr
* FamilySizeBinned_Small
* Embarked_C
* Pclass_1

Wir entscheiden uns dafür die gleichen Features wie bei KNIME herauszufiltern, um mit den gleichen Daten weiterarbeiten zu können.

In [25]:
# Aufräumen: Attribute (manuell) herausfiltern
#  (KNIME "Column Filter")
df = df.drop(['Sex_female', 'Title_Mr', 'FamilySizeBinned_Small', 'Embarked_C', 'Pclass_1'], axis=1)
# Ergebnis anzeigen
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   PassengerId             1309 non-null   Int64
 1   Survived                891 non-null    Int64
 2   KnownCabin              1309 non-null   int32
 3   Child                   1309 non-null   int32
 4   Pclass_2                1309 non-null   int32
 5   Pclass_3                1309 non-null   int32
 6   Sex_male                1309 non-null   int32
 7   Embarked_Q              1309 non-null   int32
 8   Embarked_S              1309 non-null   int32
 9   Title_Master            1309 non-null   int32
 10  Title_Miss              1309 non-null   int32
 11  Title_Mrs               1309 non-null   int32
 12  Title_Rare              1309 non-null   int32
 13  FamilySizeBinned_No     1309 non-null   int32
 14  FamilySizeBinned_Large  1309 non-null   int32
 15  FareBinned_Low       

In [26]:
# Daten wieder aufteilen
#  (KNIME: "Row Splitter")
df_train = df[df['Survived'].notna()]
df_test  = df[df['Survived'].isna()]

In [27]:
# Irrelevante Attribute filtern
#  (KNIME: "Column Filter")
# Trainingsdaten: PassengerId
df_train = df_train.drop(['PassengerId'], axis=1)
# Testdaten: Survived
df_test = df_test.drop(['Survived'], axis=1)

In [28]:
display(df_train.head())

,Survived,KnownCabin,Child,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mrs,Title_Rare,FamilySizeBinned_No,FamilySizeBinned_Large,FareBinned_Low,FareBinned_Medium,FareBinned_High,FareBinned_VeryHigh
0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0
1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,1,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0
3,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1
4,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,1,0,0


In [29]:
display(df_test.head())

,PassengerId,KnownCabin,Child,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mrs,Title_Rare,FamilySizeBinned_No,FamilySizeBinned_Large,FareBinned_Low,FareBinned_Medium,FareBinned_High,FareBinned_VeryHigh
891,892,0,0,0,1,1,1,0,0,0,0,0,1,0,1,0,0,0
892,893,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0
893,894,0,0,1,0,1,1,0,0,0,0,0,1,0,0,1,0,0
894,895,0,0,0,1,1,0,1,0,0,0,0,1,0,0,1,0,0
895,896,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0


In [30]:
# Daten als Excel-Dateien speichern
#  (KNIME: "Excel Writer")
# Trainingsdaten
df_train.to_excel('../../data/titanic/new/training_v4.xlsx', index=False)
# Testdaten
df_test.to_excel('../../data/titanic/new/test_v4.xlsx', index=False)